In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth,SpotifyClientCredentials
import pandas as pd
import time

In [12]:
sp = spotipy.Spotify(auth_manager=SpotifyOAuth(
    client_id="af58c9ca0d2544988465281053fe2157",
    client_secret="278089078f1740c6a819960e4b87f5ef",
    redirect_uri="http://localhost:8888/callback",
    scope="user-library-read"))

results = sp.current_user_saved_tracks()
for idx, item in enumerate(results['items']):
    track = item['track']
    print(f"{idx + 1}. {track['artists'][0]['name']} – {track['name']}")


1. Keane – Somewhere Only We Know
2. Djo – End of Beginning
3. MGMT – Little Dark Age
4. Coldplay – Yellow
5. Major Lazer – Lean On
6. The Weeknd – Starboy
7. Roddy Ricch – The Box
8. One Direction – Night Changes
9. The Beatles – Hey Jude - Remastered 2015
10. Arctic Monkeys – I Wanna Be Yours
11. Billie Eilish – lovely (with Khalid)
12. Coldplay – Viva La Vida
13. Adele – Set Fire to the Rain
14. Otnicka – Peaky Blinder
15. Imagine Dragons – Demons
16. Imagine Dragons – On Top Of The World
17. Coldplay – Hymn for the Weekend
18. Imagine Dragons – Bones
19. Gym Class Heroes – Stereo Hearts (feat. Adam Levine)
20. Famy – Ava


In [ ]:
SPOTIPY_CLIENT_ID = 'af58c9ca0d2544988465281053fe2157'
SPOTIPY_CLIENT_SECRET = '278089078f1740c6a819960e4b87f5ef'
SPOTIPY_REDIRECT_URI = 'http://localhost:8888/callback' 
PLAYLIST_ID = '6kq0gmy1HjhWkXlh5BGjur'
scope = "user-library-read playlist-read-private"

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=SPOTIPY_CLIENT_ID,
                                               client_secret=SPOTIPY_CLIENT_SECRET,
                                               redirect_uri=SPOTIPY_REDIRECT_URI,
                                               scope=scope))

def get_playlist_tracks(playlist_id):
    results = sp.playlist_items(playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

def get_audio_features(track_id):
    return sp.audio_features(track_id)[0]

def get_artist_followers(artist_id):
    artist_info = sp.artist(artist_id)
    return artist_info['followers']['total']

# Get playlist information
playlist = sp.playlist(PLAYLIST_ID)
playlist_name = playlist['name']
print(f"Processing playlist: {playlist_name}")

all_tracks_data = []
artist_follower_cache = {}

tracks = get_playlist_tracks(PLAYLIST_ID)

for track in tracks:
    track_info = track['track']
    if track_info is not None:  # Some tracks might be None due to availability issues
        track_id = track_info['id']
        track_name = track_info['name']
        artists = track_info['artists']
        artist_names = ", ".join([artist['name'] for artist in artists])
        album = track_info['album']['name']
        
        follower_counts = []
        for artist in artists:
            artist_id = artist['id']
            if artist_id not in artist_follower_cache:
                artist_follower_cache[artist_id] = get_artist_followers(artist_id)
                time.sleep(0.1)  # Add delay to avoid rate limiting
            follower_counts.append(artist_follower_cache[artist_id])
        
        audio_features = get_audio_features(track_id)
        if audio_features:
            track_data = {
                'Playlist': playlist_name,
                'Track Name': track_name,
                'Artist(s)': artist_names,
                'Artist Follower Count(s)': ", ".join(map(str, follower_counts)),
                'Album': album,
                'Danceability': audio_features['danceability'],
                'Energy': audio_features['energy'],
                'Key': audio_features['key'],
                'Loudness': audio_features['loudness'],
                'Mode': audio_features['mode'],
                'Speechiness': audio_features['speechiness'],
                'Acousticness': audio_features['acousticness'],
                'Instrumentalness': audio_features['instrumentalness'],
                'Liveness': audio_features['liveness'],
                'Valence': audio_features['valence'],
                'Tempo': audio_features['tempo'],
                'Duration_ms': audio_features['duration_ms'],
                'Time Signature': audio_features['time_signature']
            }
            all_tracks_data.append(track_data)
    
    # Add a small delay to avoid hitting rate limits
    time.sleep(0.1)

df = pd.DataFrame(all_tracks_data)
csv_filename = 'my_playlist.csv'
df.to_csv(csv_filename, index=False)

print(f"Data has been saved to {csv_filename}")
print(f"Total tracks processed: {len(all_tracks_data)}")

Processing playlist: ML Project


In [ ]:
'''
# Function to get artist ID from name
def get_artist_id(name):
    try:
        results=sp.search(q='artist:' + name, type='artist')
        items= results['artists']['items']
        if len(items) > 0:
            return items[0]['id']
        else:
            print(f"No artist found for name: {name}")
            return None
    except Exception as e:
        print(f"Error searching for artist {name}: {str(e)}")
        return None
if 'track_artist' not in df.columns:
    raise ValueError("The 'track_artist' column is not present in the CSV file.")
unique_artists = df['track_artist'].unique()

print(f"Total unique artists: {len(unique_artists)}")
artist_id_dict = {}

for artist_name in unique_artists:
    print(f"Processing artist: {artist_name}")
    
    artist_id = get_artist_id(artist_name)
    if artist_id:
        artist_id_dict[artist_name] = artist_id
        print(f"Successfully retrieved ID for artist: {artist_name}")
    else:
        artist_id_dict[artist_name] = 'Not Found'
        print(f"Could not retrieve ID for artist: {artist_name}")
    
    time.sleep(1)

artist_id_df = pd.DataFrame.from_dict(artist_id_dict, orient='index', columns=['artist_id'])
artist_id_df.index.name = 'artist_name'

output_file = 'artist_ids.csv'
artist_id_df.to_csv(output_file)

print(f"Successfully saved artist IDs to {output_file}")
print(f"Total artists processed: {len(artist_id_dict)}")
print(f"Artists with IDs found: {sum(1 for id in artist_id_dict.values() if id != 'Not Found')}")
print(f"Artists with IDs not found: {sum(1 for id in artist_id_dict.values() if id == 'Not Found')}")
'''


In [ ]:
SPOTIPY_CLIENT_ID ='af58c9ca0d2544988465281053fe2157'
SPOTIPY_CLIENT_SECRET ='278089078f1740c6a819960e4b87f5ef'
client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIPY_CLIENT_ID, client_secret=SPOTIPY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

def get_artist_followers(artist_id):
    try:
        artist_info = sp.artist(artist_id)
        return artist_info['followers']['total']
    except:
        return None

# Read the CSV file
input_file =r'C:\Users\DEBARSHI\Documents\Programs\Python\ML-with-Spotify\Stage 1\artist_ids.csv'
df = pd.read_csv(input_file)

# Ensure 'artist_id' column exists
if 'artist_id' not in df.columns:
    raise ValueError("The 'artist_id' column is not present in the CSV file.")

# Create a new column for follower counts
df['follower_count'] = None

# Process each artist
for index, row in df.iterrows():
    artist_id = row['artist_id']
    if artist_id != 'Not Found':
        follower_count = get_artist_followers(artist_id)
        if follower_count is not None:
            df.at[index, 'follower_count'] = follower_count
        else:
            df.at[index, 'artist_id'] = 'Not Found'
    
    # Add a small delay to avoid hitting rate limits
    time.sleep(0.1)

    # Print progress
    if (index + 1) %10 == 0:
        print(f"Processed {index + 1} artists")

# Remove rows where artist_id is 'Not Found'
df = df[df['artist_id'] != 'Not Found']

# Save the updated DataFrame back to the same CSV file
output='follower_count.csv'
df.to_csv(output, index=False)

print(f"Updated data has been saved to {output}")
print(f"Total artists processed: {len(df)}")